In [1]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

/Users/elianeorlandin/Documents/Desenvolvimento/IA-Project-Agent/llamaindex_agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
llm = Groq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [3]:
def calcular_imposto_renda(rendimento:float) -> str:
    """ 
    Calcula o imposto de renda com base no rendimento anual.
    Args:
        rendimento (float): Rendimento anual do contribuinte.
    Returns:
        str: O valor do imposto devido com base no rendimento.
    """
    if rendimento <= 2000:
        return "Isento de imposto de renda."
    elif 2000 < rendimento <= 5000:
        imposto = (rendimento - 2000) * 0.10
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    elif 5000 < rendimento <= 10000:
        imposto = (rendimento - 5000) * 0.15 + 300
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."
    else:
        imposto = (rendimento - 10000) * 0.20 + 1050
        return f"Imposto devido é de: R$ {imposto:.2f}, base em um rendimento de R$ {rendimento:.2f}."

### Convertendo Função em Ferramenta

In [4]:
from llama_index.core.tools import FunctionTool

In [5]:
ferramenta_imposto_renda = FunctionTool.from_defaults(
    fn=calcular_imposto_renda,
    name="Calcular Imposto de Renda",
    description=(
    "Calcula o imposto de renda com base no rendimento anual do contribuinte."
    "Argumento: redimento (float)."
    "Retorna o valor do imposto devido de acordo com faixas de rendimento."
    )
)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker 

In [7]:
agent_worker_imposto = FunctionCallingAgentWorker.from_tools(
    tools=[ferramenta_imposto_renda],
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)

In [8]:
from llama_index.core.agent import AgentRunner

In [9]:
agent_imposto= AgentRunner(agent_worker_imposto)

In [10]:
response = agent_imposto.chat("""
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?
"""
)

Added user message to memory: 
    Qual é o imposto de renda devido por uma pessoa com rendimento anual de R$ 7500?

=== Calling Function ===
Calling function: Calcular Imposto de Renda with args: {"rendimento": 7500}
=== Function Output ===
Imposto devido é de: R$ 675.00, base em um rendimento de R$ 7500.00.
=== LLM Response ===
O imposto de renda devido por uma pessoa com rendimento anual de R$ 7500 é de R$ 675,00.


In [11]:
import arxiv 

def consulta_artigos(titulo: str) -> str:
    """ Consulta artigos na base de dados arXiv e retorna resultados formatados."""
    busca = arxiv.Search(
        query=titulo,
        max_results=5,
        sort_by=arxiv.SortCriterion.Relevance
    )

    resultados = [
        f"Título: {artigo.title}\n"
        f"Categoria: {artigo.primary_category}\n"
        f"Link: {artigo.entry_id}\n"
        for artigo in busca.results()
    ]

    return "\n\n".join(resultados)

In [12]:
consulta_artigos_tool = FunctionTool.from_defaults(fn=consulta_artigos)

In [13]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [ferramenta_imposto_renda, consulta_artigos_tool],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [14]:
agent = AgentRunner(agent_worker)
response = agent.chat("Me retorne artigos sobre LangChain na eeducação")

Added user message to memory: Me retorne artigos sobre LangChain na eeducação
=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain na educa\u00e7\u00e3o"}


/var/folders/5z/bv195b9d0f11m9c66rw80j3r0000gp/T/ipykernel_10203/3605877752.py:15: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for artigo in busca.results()


=== Function Output ===
Título: Development and Testing of Retrieval Augmented Generation in Large Language Models -- A Case Study Report
Categoria: cs.CL
Link: http://arxiv.org/abs/2402.01733v1


Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integrated Web Application?
Categoria: cs.CR
Link: http://arxiv.org/abs/2308.01990v4


Título: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
Categoria: cs.CL
Link: http://arxiv.org/abs/2310.05421v1


Título: Poisoned LangChain: Jailbreak LLMs by LangChain
Categoria: cs.CL
Link: http://arxiv.org/abs/2406.18122v1


Título: Breast Ultrasound Report Generation using LangChain
Categoria: eess.IV
Link: http://arxiv.org/abs/2312.03013v1

=== Calling Function ===
Calling function: consulta_artigos with args: {"titulo": "LangChain e educa\u00e7\u00e3o"}
=== Function Output ===
Título: From Prompt Injections to SQL Injection Attacks: How Protected is Your LLM-Integ

### Usando o Tavily

In [15]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [16]:
tavily_key = os.environ.get("TAVILY_API_KEY")

In [17]:
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_tool = TavilyToolSpec(
    api_key=tavily_key
)

In [18]:
tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)
    

search


In [19]:
tavily_tool.search("Me rteorne artigos científicos sobre LangChain", max_results=3)

[Document(id_='29cb1e12-0732-4fcc-ba58-101ad8f4f339', embedding=None, metadata={'url': 'https://community.revelo.com.br/faca-perguntas-ao-seu-pdf-usando-langchain-llama-2-e-python/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Durante 2023, a Meta anunciou o LLaMA 2 (de agora em diante vou chamá-lo simplesmente de Llama 2), um LLM de código aberto que é a evolução de seu modelo anterior (LLaMA 1), que pode ser usado para criar aplicações para fins comerciais. Para carregar o modelo Llama 2, como aconteceu com o carregamento de PDF e Pinecode, LangChain também nos fornece uma interface (que fácil!). Isto não significa que devolva dois parágrafos ou duas palavras, mas tem a ver com os textos que o modelo considera significativos para fornecer uma resposta: e um documento pode conter vários parágrafos. O que se vê neste artigo é 

#### Definição de ferramentas externas e integração com o agente  
Criamos uma ferramenta utilizando a API do Tavily para realizar buscas sobre um determinado tópico.  
Essa ferramenta será incorporada ao agente para permitir pesquisas contextuais.  

In [22]:
from llama_index.core.tools import FunctionTool
tavily_tool_function = FunctionTool.from_defaults(
    fn=tavily_tool.search,
    name="Tavily Search",
    description=(
        "Busca artigos com Tavily sobre um determinado tópico"
    )
)

In [24]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[tavily_tool_function],
    verbose=True,
    allow_parallel_tool_calls=False,
    llm=llm
)

In [25]:
agent = AgentRunner(agent_worker)

In [26]:
response = agent.chat("Me retorne artigos sobre LLM e LangChain")

Added user message to memory: Me retorne artigos sobre LLM e LangChain
=== Calling Function ===
Calling function: Tavily Search with args: {"max_results": 6, "query": "LLM e LangChain"}
=== Function Output ===
[Document(id_='8c34e89a-924c-43fc-bd19-0d50e14e8d26', embedding=None, metadata={'url': 'https://www.techtarget.com/searchenterpriseai/tip/How-to-use-LangChain-for-LLM-application-development'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='[LangChain](https://www.techtarget.com/searchenterpriseai/definition/LangChain), an open source framework for building AI applications, has become a de facto standard for working with LLMs and integrating [APIs](https://www.techtarget.com/searchapparchitecture/definition/application-program-interface-API). The tool serves as a critical intermediary, enabling a targeted LLM to interface wi

In [27]:
print(response)

Os artigos sobre LLM e LangChain abordam a integração de modelos de linguagem grandes (LLMs) com a framework LangChain para desenvolver aplicações de inteligência artificial (IA) mais avançadas. A LangChain fornece uma estrutura para gerenciar workflows de LLMs e integrá-los com APIs, ferramentas e utilitários de software, permitindo que os desenvolvedores criem aplicações de IA mais complexas e escaláveis.

Os artigos destacam a importância da LangChain para simplificar a integração de LLMs em aplicações, fornecendo abstrações para tarefas comuns como gerenciamento de prompts, execução de chains e gerenciamento de memória. Além disso, a combinação da LangChain com bancos de dados de vetores permite a realização de buscas semânticas eficientes, permitindo que os LLMs acessem e processem grandes quantidades de dados não estruturados.

Os artigos também discutem a diferença entre LLM Chains e LLM Agent Executors, dois conceitos fundamentais na LangChain, e como eles podem ser usados para

In [28]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [29]:
url = "files/LLM.pdf"
artigo = SimpleDirectoryReader(input_files=[url]).load_data()

Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 103 0 (offset 0)
Ignoring wrong pointing object 108 0 (offset 0)
Ignoring wrong pointing object 149 0 (offset 0)
Ignoring wrong pointing object 155 0 (offset 0)
Ignoring wrong pointing object 158 0 (offset 0)
Ignoring wrong pointing object 160 0 (offset 0)
Ignoring wrong pointing object 163 0 (offset 0)
Ignori

In [31]:
url = "files/LLM_2.pdf"
tutorial = SimpleDirectoryReader(input_files=[url]).load_data()

### Gerar os Embeddings

In [32]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [36]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name= "inTfloat/multilingual-e5-large"
)

In [37]:
artigo_index = VectorStoreIndex.from_documents(artigo)
tutorial_index = VectorStoreIndex.from_documents(tutorial)

In [38]:
artigo_index.storage_context.persist(persist_dir="artigo")
tutorial_index.storage_context.persist(persist_dir="tutorial")

### Engine de Busca

In [39]:
from llama_index.core import StorageContext, load_index_from_storage    

In [40]:
storage_context = StorageContext.from_defaults(
    persist_dir="artigo"
)
artigo_index = load_index_from_storage(storage_context)

storage_context = StorageContext.from_defaults(
    persist_dir="tutorial"
)
tutorial_index = load_index_from_storage(storage_context)

In [42]:
artigo_engine = artigo_index.as_query_engine(similary_top_k=3, llm=llm)
tutorial_engine = tutorial_index.as_query_engine(similary_top_k=3, llm=llm)

In [43]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [45]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=artigo_engine,
        metadata=ToolMetadata(
            name="artigo_engine",
            description=(
                "Fornece informaçõe4s sobre LLM e LangChain."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    ),
    QueryEngineTool(
        query_engine=tutorial_engine,
        metadata=ToolMetadata(
            name="tutorial_engine",
            description=(
                "Fornece informaçõe4s sobre casos de uso e aplicações em LLMs."
                "Use uma pergunta detalhada em texto simples como entrada para a ferramenta"
            )
        )
    )
]

In [46]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    query_engine_tools,
    verbose=True,
    allow_parallel_tool_calls=True,
    llm=llm
)
agent_document = AgentRunner(agent_worker)

In [47]:
response = agent_document.chat(
    "Quais as principais aplicações posso construir com LLMs e LangChain?"
)

Added user message to memory: Quais as principais aplicações posso construir com LLMs e LangChain?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais aplica\u00e7\u00f5es posso construir com LLMs e LangChain?"}
=== Function Output ===
Com LLMs, você pode desenvolver aplicativos prontos para produção, como modelos de linguagem personalizados para atender às necessidades específicas do seu domínio. Além disso, você pode ajustar os modelos de código aberto para melhorar o desempenho em seu domínio específico. Isso inclui a capacidade de treinar os modelos com seus dados específicos, permitindo uma maior precisão e controle sobre os resultados. Com a combinação de LLMs e ferramentas como a Databricks, você pode criar soluções personalizadas para suas necessidades, desde a criação de modelos de linguagem até a implementação de soluções de dados.
=== Calling Function ===
Calling function: tutorial_engine with args: {"input": "Quais as principai

In [48]:
response = agent_document.chat(
    "Quais as principais tendências em LangChain e LLM?"
)

Added user message to memory: Quais as principais tendências em LangChain e LLM?
=== Calling Function ===
Calling function: artigo_engine with args: {"input": "Quais as principais tend\u00eancias em LangChain e LLM?"}
=== Function Output ===
Não há informações sobre tendências em LangChain e LLM. No entanto, para começar a usar LLMs, existem algumas opções disponíveis, como assistir a apresentações sob demanda, participar de cursos sobre LLMs ou explorar exemplos de código para desenvolver aplicativos prontos para produção com LLMs.
=== Calling Function ===
Calling function: tutorial_engine with args: {"input": "Quais as principais tend\u00eancias em LangChain e LLM?"}
=== Function Output ===
As principais tendências em LangChain e LLM incluem a proliferação de LLMs de código aberto, que democratizou o acesso à tecnologia de ponta de processamento de linguagem, permitindo que pesquisadores, desenvolvedores e amadores experimentassem, personalizassem e implantassem soluções de IA com um

### Agent ReAct

In [49]:
from llama_index.core.agent import ReActAgent 

In [50]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    verbose=True,
    llm=llm
)

In [51]:
response = agent.chat(
    "Quais as principais ferramentas usadas em LAngChain?"
)

> Running step d4cc8851-0099-47c0-8dd2-1a880635fed8. Step input: Quais as principais ferramentas usadas em LAngChain?
Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: artigo_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há menção às ferramentas usadas em LangChain no texto fornecido. O texto discute grandes modelos de linguagem (LLM), serviços proprietários como o ChatGPT e modelos de código aberto, mas não menciona LangChain ou suas ferramentas.
> Running step aa623f1b-2b32-4852-8e25-ab4b6e3fbf16. Step input: None
Thought: A ferramenta artigo_engine não forneceu informações suficientes sobre as principais ferramentas usadas em LangChain. Vou tentar novamente com a ferramenta tutorial_engine.
Action: tutorial_engine
Action Input: {'input': 'Quais as principais ferramentas usadas em LangChain?'}
Observation: Não há menção às principais ferramentas usadas em Lan

In [52]:
response = agent.chat(
    "Quais as principais tendências em LAngChain que eu deveria estudar?"
)

> Running step a0c4c1e6-6b3a-4252-b55a-866e30553cff. Step input: Quais as principais tendências em LAngChain que eu deveria estudar?
Thought: O usuário está procurando por tendências em LangChain. Eu posso usar o artigo_engine para obter informações sobre as principais tendências em LangChain.
Action: artigo_engine
Action Input: {'input': 'principais tendências em LangChain'}
Observation: Não há menção específica às principais tendências em LangChain no texto fornecido. O texto discute os grandes modelos de linguagem (LLM), a importância de ter controle sobre os dados e a capacidade de ajustar modelos de código aberto para melhorar o desempenho em domínios específicos, mas não aborda o tópico de LangChain.
> Running step dc43290c-1f27-4c96-b92c-032b5fcb1e51. Step input: None
Thought: O artigo_engine não forneceu informações sobre as principais tendências em LangChain. Vou tentar usar o tutorial_engine para obter informações sobre as principais tendências em LangChain.
Action: tutorial_